In [6]:
#ignore warnings
import warnings
warnings.filterwarnings("ignore")

#linear algebra
import numpy as np
import pandas as pd

#helper modules
import acquire
import prepare

#statistical tests
from scipy import stats
from scipy.stats import pearsonr, spearmanr

#visualizations
import matplotlib.pyplot as plt
import seaborn as sns

#world bank data
import wbgapi as wb

#from sklearn.model_selection import train_test_split

#display max columns
pd.set_option('display.max_columns', None)

#use this format specifier exactly AS IS to avoid breaking module, space matters!
#pd.options.display.float_format = '{:20,.2f}'.format 
#sns.set_style("white")

In [7]:
#acquire
df = acquire.get_fifa_data()

In [3]:
# #how to import function from general 

# import sys
 
# # the system path
# sys.path.append('/Users/juleszara/codeup_data_science/fifa')
 
# # importing the function
# from prepare import prepped_data

In [8]:
#prepare
df = prepare.prepped_data(df)

Before dropping nulls, 142079 rows, 111 cols
After dropping nulls. 131489 rows. 66 cols
After dropping leagues. 98804 rows. 72 cols


In [9]:
df.head()

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,club_name,league_name,nationality_id,nationality_name,club_position,position,club_joined,club_contract_valid_until,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,pace,shooting,passing,dribbling,defending,physical,crossing,finishing,heading_accuracy,short_passing,volleys,skill_dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,gk_speed,year,total_wage,age_bins,height_bins,weight_bins,year_joined,seniority,league_encoded
0,158023,L. Messi,"RW, ST, CF",93,93,78000000.0,320000.0,34,170,72,73.0,Paris Saint-Germain,French Ligue 1,52,Argentina,RW,Right Wing,2021-08-10,2023.0,Left,4,4,5,Medium/Low,85,92,91,95,34,65,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,20,35,24,6,11,15,14,8,0,2022,78320000.0,older,short,average,2021,1,4.0
1,188545,R. Lewandowski,ST,92,92,119500000.0,270000.0,32,185,81,21.0,FC Bayern München,German 1. Bundesliga,37,Poland,ST,Striker,2014-07-01,2023.0,Right,4,4,5,High/Medium,78,92,79,86,44,82,71,95,90,85,89,85,79,85,70,88,77,79,77,93,82,90,85,76,86,87,81,49,95,81,90,35,42,19,15,6,12,8,10,0,2022,119770000.0,older,medium,average,2014,8,7.0
2,20801,Cristiano Ronaldo,"ST, LW",91,91,45000000.0,270000.0,36,187,83,11.0,Manchester United,English Premier League,38,Portugal,ST,Striker,2021-08-27,2023.0,Right,4,5,5,High/Low,87,94,80,88,34,75,87,95,90,80,86,88,81,84,77,88,85,88,86,94,74,94,95,77,77,93,63,29,95,76,88,24,32,24,7,11,15,14,11,0,2022,45270000.0,older,medium,average,2021,1,2.0
3,190871,Neymar Jr,"LW, CAM",91,91,129000000.0,270000.0,29,175,68,73.0,Paris Saint-Germain,French Ligue 1,54,Brazil,LW,Left Wing,2017-08-03,2025.0,Right,5,5,5,High/Medium,91,83,86,94,37,63,85,83,63,86,86,95,88,87,81,95,93,89,96,89,84,80,64,81,53,81,63,37,86,90,93,35,32,29,9,9,15,15,11,0,2022,129270000.0,younger,medium,slim,2017,5,4.0
4,192985,K. De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,181,70,10.0,Manchester City,English Premier League,7,Belgium,RCM,Right (off-centre) Centre Midfield,2015-08-30,2025.0,Right,5,4,4,High/High,76,86,93,88,64,78,94,82,55,94,82,88,85,83,93,91,76,76,79,91,78,91,63,89,74,91,76,66,88,94,83,68,65,53,15,13,5,10,13,0,2022,125850000.0,older,medium,average,2015,7,2.0


In [4]:
#df.columns.to_list()

In [5]:
#drop columns
df = df.drop(columns = ['player_url',
                  'long_name',
                  'player_positions',
                  'club_jersey_number',
                  'club_loaned_from',
                  'nation_team_id',
                  'nation_position',
                  'nation_jersey_number',
                  'real_face',
                  'player_tags',
                  'player_traits',
                   'ls',
                   'st',
                   'rs',
                   'lw',
                   'lf',
                   'cf',
                   'rf',
                   'rw',
                   'lam',
                   'cam',
                   'ram',
                   'lm',
                   'lcm',
                   'cm',
                   'rcm',
                   'rm',
                   'lwb',
                   'ldm',
                   'cdm',
                   'rdm',
                   'rwb',
                   'lb',
                   'lcb',
                   'cb',
                   'rcb',
                   'rb',
                   'gk',
                   'player_face_url',
                   'club_logo_url',
                   'club_flag_url',
                   'nation_logo_url',
                   'nation_flag_url',
                  ])

In [6]:
def goal_keeper_stats(df):
    #add 0 values to non goal keeper players
    df['goalkeeping_speed'].fillna("0", inplace = True)
    df['defending'].fillna("0", inplace = True)
    df['physic'].fillna("0", inplace = True)
    df['dribbling'].fillna("0", inplace = True)
    df['passing'].fillna("0", inplace = True)
    df['shooting'].fillna("0", inplace = True)
    df['shooting'].fillna("0", inplace = True)
    df['pace'].fillna("0", inplace = True)
    return df

In [7]:
df = goal_keeper_stats(df)

In [17]:
def add_income_level(df):
    """This function takes the income level from the World Bank API and adds that information as a new column."""
    #select incomeLevel
    income_level = wb.economy.DataFrame()['incomeLevel']

    #select country
    country = wb.economy.DataFrame()['name']

    #turn income into df and join country
    income_level = pd.DataFrame(income_level).join(country)

    #reset index
    income_level = income_level.reset_index()

    #rename columns
    income_level.rename(columns = {'incomeLevel': 'income_level', 'name' : 'nationality_name'}, inplace = True)
    
    #inner merge of income_level
    df = pd.merge(df, income_level[['nationality_name', 'income_level']], how = 'left', on = 'nationality_name')
    
    #Add income levels to null values
    df.loc[df['nationality_name'] == 'Cape Verde Islands', ['income_level']] = 'LMC'
    df.loc[df['nationality_name'] == 'China PR', ['income_level']] = 'UMC'
    df.loc[df['nationality_name'] == 'Chinese Taipei', ['income_level']] = 'HIC'
    df.loc[df['nationality_name'] == 'Congo', ['income_level']] = 'LMC'
    df.loc[df['nationality_name'] == 'Congo DR', ['income_level']] = 'LIC'
    df.loc[df['nationality_name'] == "Côte d'Ivoire", ['income_level']] = 'LMC'
    df.loc[df['nationality_name'] == 'Egypt', ['income_level']] = 'LMC'
    df.loc[df['nationality_name'] == 'England', ['income_level']] = 'HIC'
    df.loc[df['nationality_name'] == 'Gambia', ['income_level']] = 'LIC'
    df.loc[df['nationality_name'] == 'Guinea Bissau', ['income_level']] = 'LIC'
    df.loc[df['nationality_name'] == 'Hong Kong', ['income_level']] = 'HIC'
    df.loc[df['nationality_name'] == 'Iran', ['income_level']] = 'LMC'
    df.loc[df['nationality_name'] == 'Korea DPR', ['income_level']] = 'LIC'
    df.loc[df['nationality_name'] == 'Korea Republic', ['income_level']] = 'HIC'
    df.loc[df['nationality_name'] == 'Kyrgyzstan', ['income_level']] = 'LMC'
    df.loc[df['nationality_name'] == 'Macau', ['income_level']] = 'HIC'
    df.loc[df['nationality_name'] == 'Montserrat', ['income_level']] = 'UMC'
    df.loc[df['nationality_name'] == 'Northern Ireland', ['income_level']] = 'HIC'
    df.loc[df['nationality_name'] == 'Palestine', ['income_level']] = 'LMC'
    df.loc[df['nationality_name'] == 'Republic of Ireland', ['income_level']] = 'HIC'
    df.loc[df['nationality_name'] == 'Russia', ['income_level']] = 'UMC'
    df.loc[df['nationality_name'] == 'Saint Kitts and Nevis', ['income_level']] = 'HIC'
    df.loc[df['nationality_name'] == 'Saint Vincent and the Grenadines', ['income_level']] = 'UMC'
    df.loc[df['nationality_name'] == 'Scotland', ['income_level']] = 'HIC'
    df.loc[df['nationality_name'] == 'Slovakia', ['income_level']] = 'HIC'
    df.loc[df['nationality_name'] == 'Swaziland', ['income_level']] = 'LMC'
    df.loc[df['nationality_name'] == 'Syria', ['income_level']] = 'LIC'
    df.loc[df['nationality_name'] == 'São Tomé e Príncipe', ['income_level']] = 'LMC'
    df.loc[df['nationality_name'] == 'Turkey', ['income_level']] = 'UMC'
    df.loc[df['nationality_name'] == 'Venezuela', ['income_level']] = 'LIC'
    df.loc[df['nationality_name'] == 'Wales', ['income_level']] = 'HIC'
    
    return df

In [18]:
df = add_income_level(df)

In [19]:
df.head(1)

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,club_name,league_name,nationality_id,nationality_name,club_position,position,club_joined,club_contract_valid_until,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,pace,shooting,passing,dribbling,defending,physical,crossing,finishing,heading_accuracy,short_passing,volleys,skill_dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,gk_speed,year,total_wage,age_bins,height_bins,weight_bins,year_joined,seniority,league_encoded,income_level
0,158023,L. Messi,"RW, ST, CF",93,93,78000000.0,320000.0,34,170,72,73.0,Paris Saint-Germain,French Ligue 1,52,Argentina,RW,Right Wing,2021-08-10,2023.0,Left,4,4,5,Medium/Low,85,92,91,95,34,65,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,20,35,24,6,11,15,14,8,0,2022,78320000.0,older,short,average,2021,1,4.0,UMC


In [11]:
df.isna().sum()

sofifa_id                     0
short_name                    0
overall                       0
potential                     0
value_eur                  1897
                           ... 
goalkeeping_positioning       0
goalkeeping_reflexes          0
goalkeeping_speed             0
year                          0
income_level                  8
Length: 69, dtype: int64

In [12]:
pd.set_option('display.max_rows', 183)
#see how many null values per country for variables
df.pivot_table(index = 'nationality_name',
              values = ['income_level'
                       ],
              aggfunc = lambda x:x.isna().sum())


,income_level
nationality_name,
Afghanistan,0
Albania,0
Algeria,0
Andorra,0
Angola,0
Antigua and Barbuda,0
Argentina,0
Armenia,0
Aruba,0


In [13]:
def wrangle_fifa_data(df):

    #change numerical data to integers
    df.pace = df.pace.astype(int)
    df.shooting = df.shooting.astype(int)
    df.passing = df.passing.astype(int)
    df.dribbling = df.dribbling.astype(int)
    df.defending = df.defending.astype(int)
    df.physic = df.physic.astype(int)
    df.goalkeeping_speed = df.goalkeeping_speed.astype(int)
    
    #rename columns
    df = df.rename(columns = {
                         'physic': 'physical',
                         'attacking_crossing':'crossing',
                         'attacking_finishing': 'finishing',
                         'attacking_heading_accuracy': 'heading_accuracy',
                         'attacking_short_passing': 'short_passing',
                         'attacking_volleys': 'volleys',
                         'skill_curve': 'curve',
                         'skill_fk_accuracy':'fk_accuracy',
                         'skill_long_passing': 'long_passing',
                         'skill_ball_control': 'ball_control',
                         'movement_acceleration': 'acceleration',
                         'movement_sprint_speed': 'sprint_speed',
                         'movement_agility': 'agility',
                         'movement_reactions': 'reactions',
                         'movement_balance': 'balance',
                         'power_shot_power': 'shot_power',
                         'power_jumping': 'jumping',
                         'power_stamina': 'stamina',
                         'power_strength' : 'strength',
                         'power_long_shots': 'long_shots',
                         'mentality_aggression': 'aggression',
                         'mentality_interceptions': 'interceptions',
                         'mentality_positioning': 'positioning',
                         'mentality_vision': 'vision',
                         'mentality_penalties': 'penalties',
                         'defending_marking_awareness': 'marking',
                         'defending_standing_tackle': 'standing_tackle',
                         'defending_sliding_tackle': 'sliding_tackle',
                         'goalkeeping_diving': 'gk_diving',
                         'goalkeeping_handling': 'gk_handling',
                         'goalkeeping_kicking': 'gk_kicking',
                         'goalkeeping_positioning': 'gk_positioning',
                         'goalkeeping_reflexes': 'gk_reflexes' ,
                         'goalkeeping_speed': 'gk_speed'
                              
                                                })
    
    
    #add total wage column
    df['total_wage'] = df['value_eur'] + df['wage_eur']
    
    # change columns to datetime
    #df.club_joined = pd.to_datetime(df.club_joined)
    
    #create age bins players younger than 30 are considered younger, else, older
    df['age_bins'] = pd.cut(df['age'], bins = [0, 29, np.inf], labels = ['younger', 'older'])
    
    #create height bins

    df['height_bins'] = pd.cut(df['height_cm'], bins = 3, labels = ['short', 'medium', 'tall'])

    #create weight bins
    df['weight_bins'] = pd.cut(df['weight_kg'], bins = 3, labels = ['slim', 'average', 'heavy'])
    
    #only maintain league_level 1
    df = df[df.league_level == 1.0]

    #drop league_level 
    df = df.drop(columns = ['league_level'])
    
    df = df.dropna()
    
    #expand the club_joined to get only the year from YYYY-MM-DD
    df['club_joined'] = df.club_joined.astype('str')
    df['year_joined'] = df.club_joined.str.split('-', expand = True)[0]
    
    #change joined year to int
    df.year_joined = df.year_joined.astype(int)
    
    #add seniority column
    df['seniority'] = df.year - df.year_joined
    
    #filter by year
    values = [2017, 2018, 2019]
    df = df.loc[df['year'].isin(values)]
    
    
    
    return df

In [14]:
df = wrangle_fifa_data(df)

In [15]:
df.head()

,sofifa_id,short_name,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,club_position,club_joined,club_contract_valid_until,nationality_id,nationality_name,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physical,crossing,finishing,heading_accuracy,short_passing,volleys,skill_dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,mentality_composure,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,gk_speed,year,income_level,total_wage,age_bins,height_bins,weight_bins,year_joined,seniority
56666,20801,Cristiano Ronaldo,94,94,77000000.0,400000.0,33,1985-02-05,187,83,45.0,Juventus,Italian Serie A,ST,2018-07-10,2022.0,38,Portugal,Right,4,5,5,High/Low,Unique,127100000.0,90,93,81,89,35,79,84,94,89,81,87,88,81,76,77,94,89,91,87,96,70,95,95,88,79,93,63,29,95,82,85,95.0,28,31,23,7,11,15,14,11,0,2019,HIC,77400000.0,older,medium,average,2018,1
56667,158023,L. Messi,94,94,110500000.0,575000.0,31,1987-06-24,170,72,241.0,FC Barcelona,Spain Primera Division,RW,2004-07-01,2021.0,52,Argentina,Left,4,4,5,Medium/Medium,Unique,226500000.0,88,91,88,96,32,61,77,95,70,90,86,97,93,94,87,96,91,86,91,95,95,85,68,72,59,94,48,22,94,94,75,96.0,33,28,26,6,11,15,14,8,0,2019,UMC,111075000.0,older,short,average,2004,15
56668,190871,Neymar,92,93,118500000.0,300000.0,26,1992-02-05,175,68,73.0,Paris Saint-Germain,French Ligue 1,LW,2017-08-03,2022.0,54,Brazil,Right,5,5,5,High/Medium,Unique,228100000.0,92,84,83,95,32,59,79,87,62,84,84,96,88,87,78,95,94,90,96,94,84,80,61,81,49,82,56,36,89,87,81,94.0,27,24,33,9,9,15,15,11,0,2019,UMC,118800000.0,younger,medium,slim,2017,2
56669,155862,Sergio Ramos,91,91,51000000.0,375000.0,32,1986-03-30,184,82,243.0,Real Madrid CF,Spain Primera Division,LCB,2005-08-01,2020.0,45,Spain,Right,3,3,4,High/Medium,Normal (170-185),104600000.0,75,63,71,71,91,84,66,60,91,78,66,61,73,67,72,84,76,75,78,85,66,79,93,84,83,55,84,90,52,63,68,80.0,89,92,91,11,8,9,7,11,0,2019,HIC,51375000.0,older,medium,average,2005,14
56670,176580,L. Suárez,91,91,80000000.0,450000.0,31,1987-01-24,182,86,241.0,FC Barcelona,Spain Primera Division,ST,2014-07-11,2021.0,60,Uruguay,Right,4,3,5,High/Medium,Normal (170-185),164000000.0,80,90,79,88,52,85,77,94,77,82,88,87,86,84,64,90,86,75,82,93,83,86,69,90,83,85,87,41,92,84,85,85.0,62,45,38,27,25,31,33,37,0,2019,HIC,80450000.0,older,medium,average,2014,5


In [ ]:
df.year.dtype

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.

#### Create Outlier Function

In [ ]:
#DO NOT INCLUDE Overall or Potential

In [ ]:
#create a df w/ outlier function, and w/o outliers

#### Created encoding function

In [ ]:
# encoding: league_name, club_position, preferred_foot, work_rate, body_type, income_level, 
# age_bins, weight_bins

In [17]:
df['league_encoded'] = df.league_name.map({'Italian Serie A': 1, 
                                           'Spain Primera Division': 2,
                                          'French Ligue 1': 3
                                          })

#.astype(int)

In [18]:
df.head()

,sofifa_id,short_name,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,club_position,club_joined,club_contract_valid_until,nationality_id,nationality_name,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physical,crossing,finishing,heading_accuracy,short_passing,volleys,skill_dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,mentality_composure,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,gk_speed,year,income_level,total_wage,age_bins,height_bins,weight_bins,year_joined,seniority,league_encoded
56666,20801,Cristiano Ronaldo,94,94,77000000.0,400000.0,33,1985-02-05,187,83,45.0,Juventus,Italian Serie A,ST,2018-07-10,2022.0,38,Portugal,Right,4,5,5,High/Low,Unique,127100000.0,90,93,81,89,35,79,84,94,89,81,87,88,81,76,77,94,89,91,87,96,70,95,95,88,79,93,63,29,95,82,85,95.0,28,31,23,7,11,15,14,11,0,2019,HIC,77400000.0,older,medium,average,2018,1,1.0
56667,158023,L. Messi,94,94,110500000.0,575000.0,31,1987-06-24,170,72,241.0,FC Barcelona,Spain Primera Division,RW,2004-07-01,2021.0,52,Argentina,Left,4,4,5,Medium/Medium,Unique,226500000.0,88,91,88,96,32,61,77,95,70,90,86,97,93,94,87,96,91,86,91,95,95,85,68,72,59,94,48,22,94,94,75,96.0,33,28,26,6,11,15,14,8,0,2019,UMC,111075000.0,older,short,average,2004,15,2.0
56668,190871,Neymar,92,93,118500000.0,300000.0,26,1992-02-05,175,68,73.0,Paris Saint-Germain,French Ligue 1,LW,2017-08-03,2022.0,54,Brazil,Right,5,5,5,High/Medium,Unique,228100000.0,92,84,83,95,32,59,79,87,62,84,84,96,88,87,78,95,94,90,96,94,84,80,61,81,49,82,56,36,89,87,81,94.0,27,24,33,9,9,15,15,11,0,2019,UMC,118800000.0,younger,medium,slim,2017,2,3.0
56669,155862,Sergio Ramos,91,91,51000000.0,375000.0,32,1986-03-30,184,82,243.0,Real Madrid CF,Spain Primera Division,LCB,2005-08-01,2020.0,45,Spain,Right,3,3,4,High/Medium,Normal (170-185),104600000.0,75,63,71,71,91,84,66,60,91,78,66,61,73,67,72,84,76,75,78,85,66,79,93,84,83,55,84,90,52,63,68,80.0,89,92,91,11,8,9,7,11,0,2019,HIC,51375000.0,older,medium,average,2005,14,2.0
56670,176580,L. Suárez,91,91,80000000.0,450000.0,31,1987-01-24,182,86,241.0,FC Barcelona,Spain Primera Division,ST,2014-07-11,2021.0,60,Uruguay,Right,4,3,5,High/Medium,Normal (170-185),164000000.0,80,90,79,88,52,85,77,94,77,82,88,87,86,84,64,90,86,75,82,93,83,86,69,90,83,85,87,41,92,84,85,85.0,62,45,38,27,25,31,33,37,0,2019,HIC,80450000.0,older,medium,average,2014,5,2.0


In [10]:
df.club_position.unique()

array(['RW', 'ST', 'LW', 'RCM', 'GK', 'CF', 'CDM', 'LCB', 'RDM', 'RS',
       'LCM', 'SUB', 'CAM', 'RCB', 'LDM', 'LB', 'RB', 'LM', 'RM', 'CB',
       'LS', 'RES', 'RWB', 'LWB', 'LAM', 'LF', 'RAM'], dtype=object)

In [11]:
df.work_rate.unique()

array(['Medium/Low', 'High/Medium', 'High/Low', 'High/High',
       'Medium/Medium', 'Medium/High', 'Low/High', 'Low/Medium',
       'Low/Low'], dtype=object)

In [12]:
df.preferred_foot.unique()

array(['Left', 'Right'], dtype=object)

In [14]:
df.body_type.unique()

AttributeError: 'DataFrame' object has no attribute 'bodytype'

In [20]:
df.income_level.unique()

array(['UMC', 'HIC', 'LMC', 'LIC'], dtype=object)

In [24]:
def get_encoded(df):
    df['club_position'] = df.league_name.map({'RW': 1, 
                                              'ST': 2, 
                                              'LW': 3, 
                                              'RCM': 4,
                                              'GK': 5, 
                                              'CF': 6, 
                                              'CDM':7, 
                                              'LCB': 8, 
                                              'RDM': 9, 
                                              'RS':10,
                                              'LCM':11, 
                                              'SUB':12, 
                                              'CAM':13, 
                                              'RCB':14, 
                                              'LDM':15, 
                                              'LB':16, 
                                              'RB':17, 
                                              'LM':18, 
                                              'RM':19, 
                                              'CB':20,
                                              'LS':21, 
                                              'RES':22, 
                                              'RWB':23, 
                                              'LWB':24, 
                                              'LAM':25, 
                                              'LF':26, 
                                              'RAM':27
                                          })
    
    df['work_rate_encoded'] = df.work_rate.map({'Low/Low':1,
                                                'Low/Medium':2,
                                                'Low/High':3,
                                                'Medium/Low':4,
                                                'Medium/Medium':5,
                                                'Medium/High':6,
                                                'High/Low':7,
                                                'High/Medium':8,
                                                'High/High':9
                                                
                                          })
    df['preferred_foot_encoded'] = df.preferred_foot.map({ 'Left': 1,
                                                          'Right':2
        
    })

    
    df['income_level_encoded'] = df.income_level.map({ 'LIC':1,
                                                      'LMC':2,
                                                      'UMC':3,
                                                      'HIC':4
        
        
    })
    
    df['age_bins_encoded'] = df.age_bins.map({ 'older':1,
                                                      'younger':2
        
        
    })
    
    df['weight_bins_encoded'] = df.weight_bins.map({ 'slim':1,
                                                      'average':2,
                                                    'heavy': 3
        
    })
    
    
#     df['body_type_encoded'] = df.
    return df

In [25]:
df = get_encoded(df)

In [26]:
df.head()

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,club_name,league_name,nationality_id,nationality_name,club_position,position,club_joined,club_contract_valid_until,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,pace,shooting,passing,dribbling,defending,physical,crossing,finishing,heading_accuracy,short_passing,volleys,skill_dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,gk_speed,year,total_wage,age_bins,height_bins,weight_bins,year_joined,seniority,league_encoded,income_level,work_rate_encoded,preferred_foot_encoded,income_level_encoded,age_bins_encoded,weight_bins_encoded
0,158023,L. Messi,"RW, ST, CF",93,93,78000000.0,320000.0,34,170,72,73.0,Paris Saint-Germain,French Ligue 1,52,Argentina,NaN,Right Wing,2021-08-10,2023.0,Left,4,4,5,Medium/Low,85,92,91,95,34,65,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,20,35,24,6,11,15,14,8,0,2022,78320000.0,older,short,average,2021,1,4.0,UMC,4,1,3,1,2
1,188545,R. Lewandowski,ST,92,92,119500000.0,270000.0,32,185,81,21.0,FC Bayern München,German 1. Bundesliga,37,Poland,NaN,Striker,2014-07-01,2023.0,Right,4,4,5,High/Medium,78,92,79,86,44,82,71,95,90,85,89,85,79,85,70,88,77,79,77,93,82,90,85,76,86,87,81,49,95,81,90,35,42,19,15,6,12,8,10,0,2022,119770000.0,older,medium,average,2014,8,7.0,HIC,8,2,4,1,2
2,20801,Cristiano Ronaldo,"ST, LW",91,91,45000000.0,270000.0,36,187,83,11.0,Manchester United,English Premier League,38,Portugal,NaN,Striker,2021-08-27,2023.0,Right,4,5,5,High/Low,87,94,80,88,34,75,87,95,90,80,86,88,81,84,77,88,85,88,86,94,74,94,95,77,77,93,63,29,95,76,88,24,32,24,7,11,15,14,11,0,2022,45270000.0,older,medium,average,2021,1,2.0,HIC,7,2,4,1,2
3,190871,Neymar Jr,"LW, CAM",91,91,129000000.0,270000.0,29,175,68,73.0,Paris Saint-Germain,French Ligue 1,54,Brazil,NaN,Left Wing,2017-08-03,2025.0,Right,5,5,5,High/Medium,91,83,86,94,37,63,85,83,63,86,86,95,88,87,81,95,93,89,96,89,84,80,64,81,53,81,63,37,86,90,93,35,32,29,9,9,15,15,11,0,2022,129270000.0,younger,medium,slim,2017,5,4.0,UMC,8,2,3,2,1
4,192985,K. De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,181,70,10.0,Manchester City,English Premier League,7,Belgium,NaN,Right (off-centre) Centre Midfield,2015-08-30,2025.0,Right,5,4,4,High/High,76,86,93,88,64,78,94,82,55,94,82,88,85,83,93,91,76,76,79,91,78,91,63,89,74,91,76,66,88,94,83,68,65,53,15,13,5,10,13,0,2022,125850000.0,older,medium,average,2015,7,2.0,HIC,9,2,4,1,2


In [27]:
df.isna().sum()

sofifa_id                 0
short_name                0
player_positions          0
overall                   0
potential                 0
                         ..
work_rate_encoded         0
preferred_foot_encoded    0
income_level_encoded      0
age_bins_encoded          0
weight_bins_encoded       0
Length: 78, dtype: int64